In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from keras.preprocessing.image import ImageDataGenerator

In [10]:
metadata_path = './skin-cancer-mnist-ham10000/HAM10000_metadata.csv'
metadat = pd.read_csv(metadata_path)
num_samples = len(metadat)
metadat = metadat.sample(frac=1,random_state=12).reset_index(drop=True) #shuffling data

#metadat = metadat['image_id'].add('.jpg')
metadat.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0003559,ISIC_0025810,bkl,consensus,75.0,male,face
1,HAM_0004240,ISIC_0032517,mel,histo,50.0,male,back
2,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen
3,HAM_0003229,ISIC_0031140,bcc,histo,60.0,male,chest
4,HAM_0005963,ISIC_0028129,nv,follow_up,45.0,male,foot


In [11]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 224, 224
train_data_dir = "./train"
val_data_dir = "./val"
batch_size = 1

vgg = applications.vgg19.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [24]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
shear_range=0.2
)

val_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
shear_range=0.2
)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical", 
shuffle = True, 
seed = 12
)

val_generator = val_datagen.flow_from_directory(
val_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical", 
shuffle = True, 
seed = 12
)

Found 6408 images belonging to 7 classes.
Found 1602 images belonging to 7 classes.


In [25]:
train_feat = []
train_lbl  = []

val_feat   = []
val_lbl    = []

In [26]:
for i in range(len(train_generator)):
    nxt_data = train_generator.next()
    feature = vgg.predict(nxt_data[0])
    train_feat.append(feature.flatten())
    train_lbl.append(nxt_data[1])

In [27]:
for i in range(len(val_generator)):
    nxt_data = val_generator.next()
    feature = vgg.predict(nxt_data[0])
    val_feat.append(feature.flatten())
    val_lbl.append(nxt_data[1])

In [28]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=7, random_state=0).fit(train_feat)

In [29]:
kmeans_val_pred = kmeans.predict(val_feat)

In [30]:
train_lbl = [np.where(i==1)[0][0] for i in train_lbl]
val_lbl = [np.where(i==1)[0][0] for i in val_lbl]

In [31]:
print(sum(val_lbl == kmeans_val_pred)/len(kmeans_val_pred))

0.07240948813982521
